In [1]:
spark.conf.set('spark.sql.repl.eagerEval.enabled', True)

In [4]:
%load_ext sparksql_magic

# Product Orders Problem Statement

You are a data engineer for a retail company that sells products online and in physical stores. The company has provided you with the following database schema:

    orders (order_id, order_date, fulfillment_type)
    order_items (order_id, product_id, quantity, price)
    products (product_id, product_name, category)

The retail company is interested in analyzing their sales data to identify trends in customer behavior and product performance. They have asked you to write a SQL query to find the top 10 best-selling products by revenue for online orders in the "Electronics" category, along with the total revenue generated by each product.

Write a SQL/PySpark query to answer this question.

In [8]:
%%sparksql
SELECT 
    p.product_name, 
    SUM(oi.quantity * oi.price) AS revenue
FROM 
    orders o 
    JOIN order_items oi ON o.order_id = oi.order_id 
    JOIN products p ON oi.product_id = p.product_id
WHERE 
    o.fulfillment_type = 'Online' 
    AND p.category = 'Electronics'
GROUP BY 
    p.product_name
ORDER BY 
    revenue DESC
LIMIT 10;

product_name,revenue
Laptop,480.0
Smartphone,300.0
TV,240.0
Headphones,150.0


In [7]:
from pyspark.sql.functions import col
from pyspark.sql.functions import sum

orders_df = spark.table("orders")
products_df = spark.table("products")
order_items_df = spark.table("order_items")

# join the tables
joined_df = orders_df.join(order_items_df, orders_df.order_id == order_items_df.order_id).join(products_df, order_items_df.product_id == products_df.product_id)

# filter the joined table
filtered_df = joined_df.filter((orders_df.fulfillment_type == "Online") & (products_df.category == "Electronics"))

# aggregate the data
agg_df = filtered_df.groupBy("product_name").agg(sum(col("quantity") * col("price")).alias("revenue"))

# sort and limit the data
result_df = agg_df.orderBy("revenue", ascending=False).limit(10)

# display the result
result_df

product_name,revenue
Laptop,480.0
Smartphone,300.0
TV,240.0
Headphones,150.0
